### Properties

We have seen how we can just assign and read attribute values directly on an object:

In [1]:
import math

In [2]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    def area(self):
        return math.pi * self.radius ** 2

In [3]:
c = Circle(1)

In [4]:
c.radius

1

In [5]:
c.area()

3.141592653589793

And of course, we can change the radius on that object directly:

In [6]:
c.radius = 2

In [7]:
c.radius

2

In [8]:
c.area()

12.566370614359172

One undesirable thing here is that we can set a negative value when we create the `Circle` instance.

In [9]:
c = Circle(-1)

In [10]:
c.radius, c.area()

(-1, 3.141592653589793)

Having a negative radius does not make much sense - and in fact, we can set the radius to a string even:

In [11]:
c = Circle('100')

In [12]:
c.radius

'100'

And our `area()` method will fail when called:

In [13]:
try:
    c.area()
except TypeError as ex:
    print(ex)

unsupported operand type(s) for ** or pow(): 'str' and 'int'


We can fix this by doing some checks in the `__init__` method:

In [14]:
class Circle:
    def __init__(self, radius):
        if not (isinstance(radius, float) or isinstance(radius, int)):
            raise ValueError('radius must be an integer or a float')
        if radius < 0:
            raise ValueError('radius cannot be negative')
        self.radius = radius
        
    def area(self):
        return math.pi * self.radius ** 2

In [15]:
c = Circle(1)

In [16]:
c = Circle(1.5)

In [17]:
try:
    c = Circle('100')
except ValueError as ex:
    print(ex)

radius must be an integer or a float


In [18]:
try:
    c = Circle(-10)
except ValueError as ex:
    print(ex)

radius cannot be negative


So that fixes that problem. But now, we also have the problem that we can change the radius, **after** the instance has been created, to something unacceptable:

In [19]:
c = Circle(1)

In [20]:
c.radius = 'hello'

Because the `radius` arttribute is just a direct attribute on the instance there's not much we can do about that. (Also, if you have experience in other programming languages that have something called a private scope, there is nothing like that in Python - everything is public).

Python provides an alternative mechanism to these "bare" attributes - something called **properties**.

Properties are defined using functions for **setting** the property value, and **getting** the property value.

Additionally, we can define properties that have only a getter (so a read-only property), or, far less frequently, only a getter (so a write-only property), which we won't cover here.

Let's start with read-only properties first.

In [21]:
class Circle:
    def __init__(self, radius):
        if not (isinstance(radius, float) or isinstance(radius, int)):
            raise ValueError('radius must be an integer or a float')
        if radius < 0:
            raise ValueError('radius cannot be negative')
        self._radius = radius
        
    @property
    def radius(self):
        print('radius getter called...')
        return self._radius

Note that we still store the value for the radius in a "bare" attribute in the class - but we use the **convention** of prefixing the attribute name with an underscore - this is meant to indicate (by **convention**) that the attribute is "private" to the implementation of the class - anyone using our class will understand they should not modify that attribute directly (but they still can if they want to - however, we're not to blame if they mess something up! :-) )

Note that when the `property` **decorator** is applied to an instance method - the **name of the method** will define the **name of the property**.

In [22]:
c = Circle(10)

In [23]:
c.radius

radius getter called...


10

By the way, we can use a read-only property for `area` as well - that allows us to use `c.area` instead of `c.area()` - not a hude deal, but nicer syntax:

In [24]:
class Circle:
    def __init__(self, radius):
        if not (isinstance(radius, float) or isinstance(radius, int)):
            raise ValueError('radius must be an integer or a float')
        if radius < 0:
            raise ValueError('radius cannot be negative')
        self._radius = radius
        
    @property
    def radius(self):
        print('radius getter called...')
        return self._radius
    
    @property
    def area(self):
        print('property area called...')
        return math.pi * self.radius ** 2

Note how I used `self.radius` in the `area` property - this will actually call the property getter for `radius`. We could have used `self._radius` instead, but since we have a property getter, we generally tend to use the property itself, rather than the "backing" variable.

In [25]:
c = Circle(1)

In [26]:
c.area

property area called...
radius getter called...


3.141592653589793

So now we essentially have an "immutable" class - we can set the radius when we create an instance, but there's nothing that allows us to change the radius property **after** the instance has been created. (although technically this is not true, since we can always modify `_radius` directly - but we're not supposed to, because of that leading underscore).

What if we want to allow users to modify the radius?

For that, we can write a property **setter**. Again we use a decorator and an instance method to do so.

In [27]:
class Circle:
    def __init__(self, radius):
        if not (isinstance(radius, float) or isinstance(radius, int)):
            raise ValueError('radius must be an integer or a float')
        if radius < 0:
            raise ValueError('radius cannot be negative')
        self._radius = radius
        
    @property
    def radius(self):
        print('radius getter called...')
        return self._radius
    
    @radius.setter
    def radius(self, value):
        print('radius setter called...')
        if not (isinstance(value, float) or isinstance(value, int)):
            raise ValueError('radius must be an integer or a float')
        if value < 0:
            raise ValueError('radius cannot be negative')
        self._radius = value
        
    @property
    def area(self):
        print('property area called...')
        return math.pi * self.radius ** 2

Note the decorator syntax - it **must** use the property name we used for the property **setter**, and the function name **must** be the same name as well.

In [28]:
c = Circle(1)

In [29]:
c.radius = 2

radius setter called...


In [30]:
c.radius

radius getter called...


2

And now, we are protected from setting a bad value for `radius` after the instance has been created:

In [31]:
try:
    c.radius = "100"
except ValueError as ex:
    print(ex)

radius setter called...
radius must be an integer or a float


You'll notice that we have the same validation code in both the `__init__` and the radius setter.

We can get rid of the checks in `__init__` if we simply use the property setter in `__init__`:

In [32]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    @property
    def radius(self):
        print('radius getter called...')
        return self._radius
    
    @radius.setter
    def radius(self, value):
        print('radius setter called...')
        if not (isinstance(value, float) or isinstance(value, int)):
            raise ValueError('radius must be an integer or a float')
        if value < 0:
            raise ValueError('radius cannot be negative')
        self._radius = value
        
    @property
    def area(self):
        print('property area called...')
        return math.pi * self.radius ** 2

See how we used the property setter in the `__init__` method by using `self.radius = radius` - this actually calls the setter:

In [33]:
c = Circle(10)

radius setter called...


And this means our init is still protected:

In [34]:
try:
    c = Circle(-100)
except ValueError as ex:
    print(ex)

radius setter called...
radius cannot be negative


One word of caution:

The following is a mistake often made when starting to use properties:

In [35]:
class Circle:
    def __init__(self, radius):
        self._radius = radius
        
    @property
    def radius(self):
        return self.radius

Notice how `self.radius` is used **inside** the `radius` getter function?

What method is called when we read `c.radius`?

...the `radius()` method.

So, when we write `self.radius` inside the `radius()` method, we are actually calling the `radius()` method again and again and again - this is something called infinite recursion.

Let's see what happens:

In [36]:
c = Circle(1)

In [37]:
c.radius

RecursionError: maximum recursion depth exceeded

So, if you ever see this exception when you use the getter (or setter), then you are most likely calling the getter from within the getter, or the setter from within the setter which would look something like this:

In [38]:
class Circle:
    def __init__(self, radius):
        self._radius = radius
        
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, value):
        self.radius = value  # this actually calls the setter!

In [39]:
c = Circle(1)

In [40]:
c.radius

1

In [41]:
c.radius = 2

RecursionError: maximum recursion depth exceeded